In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pycaret.classification import *

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from pycaret.regression import *
from sklearn.metrics import silhouette_samples, silhouette_score
import math
import random
import matplotlib.ticker as mtick
import re

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

from neighborhoods import admin_wards
from neighborhoods import district_neighborhoods

In [2]:
random.seed(42)

In [3]:
# Load the dataset
df = pd.read_csv('../data/raw/additional_data.csv')
df.shape

(15975, 9)

In [4]:
def drop_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [5]:
df = df.drop_duplicates()
df = df.replace({True: 1, False: 0})
df['district'] = df['district'].replace('Arganzuela', 'arganzuela')

df = drop_outliers_iqr(df, 'price')


In [6]:
def has_weird_characters(input_string):
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    return bool(pattern.search(input_string))

def replace_str_values(df, column, input_tuple_list):
    for item in input_tuple_list:
        df[column] = df[column].str.replace(item[0], item[1])
    return df


In [7]:
input_tuple_list = [('Virgen del Cortijo', 'Valdefuentes'), ('Tablas', 'Valverde'), ('Tablas', 'Valverde'), ('Sanchinarro', 'Valdefuentes'), ('Montecarmelo', 'Mirasierra'), ('Arroyo del Fresno', 'Mirasierra'), ('Buena Vista', 'Buenavista'), ('Ambroz', 'Casco Histórico de Vicálvaro'), ('Águilas', 'Las Águilas'), ('-', ' '), ('Ã³', 'ó'), ('Ã¼', 'ü'),  ('Ã©', 'é'), ('Ã±', 'ñ'), ('Ã\xad', 'í'), ('Ã\x81', 'Á'), ('Ãº', 'ú'),  ('Ã¡', 'á')]

In [8]:
df = replace_str_values(df, 'neighborhood', input_tuple_list)


df['neighborhood'] = df['neighborhood'].replace('Chalet en Águilas', 'Las Águilas')
df['neighborhood'] = df['neighborhood'].replace('Águilas', 'Las Águilas')

for item in df['neighborhood'].values:
    for barrio in admin_wards:
        if barrio in item:
            df['neighborhood'] = df['neighborhood'].replace(item, barrio)

In [9]:
not_barrios = []


for item in df['neighborhood'].values:
    if item not in admin_wards:
        not_barrios.append(item)


set(not_barrios), len(set(df['neighborhood'].values))

({'Los Berrocales', 'Palomeras sureste', 'Pau de Carabanchel'}, 129)

In [10]:
df.to_csv('../data/processed/additional_data_cleaned_ER.csv', index=False)